In [1]:
import hail as hl
# Initialize Hail 
hl.init(default_reference = 'GRCh38')

Running on Apache Spark version 3.1.3
SparkUI available at http://ibd-exome-qc-m.c.daly-ibd.internal:36127
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.99-57537fea08d4
LOGGING: writing to /home/hail/hail-20230724-0148-0.2.99-57537fea08d4.log


In [2]:
mt = hl.read_matrix_table("gs://ibd-exomes-gnomad-subset/QC_round2/2.variant_narrowing/variant_narrowed.mt")
mt.describe()
# mt.show(20)
# mt.count()

----------------------------------------
Global fields:
    None
----------------------------------------
Column fields:
    's': str
----------------------------------------
Row fields:
    'locus': locus<GRCh38>
    'alleles': array<str>
    'rsid': str
    'info': struct {
        AC_raw: array<int32>, 
        AN_raw: int32, 
        AF_raw: array<float64>, 
        nhomalt_raw: array<int32>, 
        AC: array<int32>, 
        AN: int32, 
        AF: array<float64>, 
        nhomalt: array<int32>
    }
    'a_index': int32
    'was_split': bool
    'nfe_ht': struct {
        freq: array<struct {
            AC: int32, 
            AF: float64, 
            AN: int32, 
            homozygote_count: int32
        }>, 
        age_hist_het: array<struct {
            bin_edges: array<float64>, 
            bin_freq: array<int64>, 
            n_smaller: int64, 
            n_larger: int64
        }>, 
        age_hist_hom: array<struct {
            bin_edges: array<float64>, 
      

In [3]:
# #gnomad quality filtering - read in tables######################
# print("Reading in gNOMAD tables...")

# gnomad_exomes_v2_ht = hl.read_table("gs://gcp-public-data--gnomad/release/2.1.1/liftover_grch38/ht/exomes/gnomad.exomes.r2.1.1.sites.liftover_grch38.ht")
# gnomad_genomes_v2_ht = hl.read_table("gs://gcp-public-data--gnomad/release/2.1.1/liftover_grch38/ht/genomes/gnomad.genomes.r2.1.1.sites.liftover_grch38.ht")
# # print("Annotating gNOMAD exomes and genomes fields...")
# mt = mt.annotate_rows(gnomad_exomes = gnomad_exomes_v2_ht[mt.row_key])
# mt = mt.annotate_rows(gnomad_genomes = gnomad_genomes_v2_ht[mt.row_key])

# mt = mt.annotate_rows(GE_Filter = (mt.gnomad_exomes.filters.length() > 0) | mt.gnomad_exomes.filters.contains("RF"))
# mt = mt.annotate_rows(GG_Filter = (mt.gnomad_genomes.filters.length() > 0) | mt.gnomad_genomes.filters.contains("RF"))

# # print("Filtering for PASSing variants...")
# mt = mt.filter_rows(mt.GG_Filter | mt.GE_Filter, keep=False)

In [4]:
# Set GQ to missing if low
mt = mt.annotate_entries(GT = hl.or_missing(mt.GQ > 20, mt.GT))
mt = hl.variant_qc(mt, name='variant_qc')
mt.count()



(25105651, 198597)

In [13]:
# mt.variant_qc.call_rate.show(500)
mt.variant_qc.call_rate.export('gs://ibd-exomes-gnomad-subset/QC_round2/low_call_rate_vars.tsv.gz')
# cr_hist = mt.aggregate_entries(hl.expr.aggregators.hist(mt.variant_qc.call_rate, 0, 1, 30))
# p = hl.plot.histogram(cr_hist)
# show(p)

2023-07-24 05:23:08 Hail: INFO: merging 47961 files totalling 52.4M...
2023-07-24 05:24:05 Hail: INFO: while writing:
    gs://ibd-exomes-gnomad-subset/QC_round2/low_call_rate_vars.tsv.gz
  merge time: 56.732s


In [ ]:
## ---------------------------------------------- variant QC --------------------------------------------------
# Set GQ to missing if low
mt = mt.annotate_entries(GT = hl.or_missing(mt.GQ > 20, mt.GT))
mt = hl.variant_qc(mt, name='variant_qc')
mt = mt.filter_rows(mt.variant_qc.call_rate > 0.95, keep = True)
# mt = mt.filter_rows(mt.variant_qc.AC[0] > 0, keep = True)
mt = mt.filter_rows(mt.variant_qc.gq_stats.mean > 10, keep = True)

## ---------------------------------------------- sample QC --------------------------------------------------
mt = hl.sample_qc(mt, name='sample_qc')
mt = mt.filter_cols(mt.sample_qc.n_singleton < 500, keep = True)
mt = mt.filter_cols(mt.sample_qc.gq_stats.mean > 30, keep = True)

In [4]:
mt = mt.write("gs://ibd-exomes-gnomad-subset/QC_round2/3.variant+sampleQC/variant_sample_QCed.mt", overwrite=True)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

